In [1]:
!pip install langchain langchain_community langchain_openai sentence-transformers

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install keras tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install langchain_google_genai faiss-cpu sentence_transformers

Defaulting to user installation because normal site-packages is not writeable


In [4]:
# 라이브러리 불러오기
import os
import sys
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
import os
from dotenv import load_dotenv
load_dotenv()
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import TextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
import faiss
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
import pickle
import ast
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from tqdm import tqdm
from collections import defaultdict
import re
from joblib import Parallel, delayed

In [5]:
# 임베딩과 벡터스토어 불러오기
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
new_vectorstore = FAISS.load_local("vectorstore_index_contextual_nonoverlapping", embeddings, allow_dangerous_deserialization=True)
new_vectorstore

/tmp/ipykernel_369559/3790377730.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/home/rlaalsduf/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-19 16:06:25.314707: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn the

In [11]:
import json
# JSON 파일에서 클러스터 매핑 불러오기
def load_cluster_mapping(filename="for_header_ex/cluster_mapping_header.json"):
    with open(filename, "r", encoding="utf-8") as f:
        cluster_mapping = json.load(f)
    return cluster_mapping

# 클러스터 매핑 불러오기
cluster_mapping = load_cluster_mapping()

In [10]:
import numpy as np
import ast
import pickle

# 그래프 임베딩 데이터 불러오기
data = pd.read_csv("for_header_ex/user_with_chunk_embeddings.csv")

# 'chunk_header_embedding' 컬럼의 각 값을 문자열에서 리스트로 변환한 후 numpy 배열로 변환
data['chunk_header_embedding'] = data['chunk_header_embedding'].apply(lambda x: np.array(ast.literal_eval(x)))

data['movie_explain'] = data['movie_explain'].apply(ast.literal_eval)

# RAPTOR 트리 불러오기
with open("for_header_ex/text_header_raptor_tree.pkl", "rb") as f:
    loaded_raptor_tree = pickle.load(f)

In [9]:
loaded_raptor_tree[4]

,user_id,embedding,cluster,metadata
0,cluster_4_0,"[0.023921264, -0.1083487, 0.01691085, 0.021923...",2,"{'level': 4, 'origin': 'summary_of_cluster_0_l..."
1,cluster_4_6,"[0.020734997, -0.076683186, 0.00592301, 0.0002...",2,"{'level': 4, 'origin': 'summary_of_cluster_6_l..."
2,cluster_4_3,"[0.01045218, -0.09917677, -0.009026413, 0.0223...",2,"{'level': 4, 'origin': 'summary_of_cluster_3_l..."
3,cluster_4_15,"[0.0117771225, -0.10302591, 0.0030428185, 0.01...",0,"{'level': 4, 'origin': 'summary_of_cluster_15_..."
4,cluster_4_2,"[0.00651194, -0.08133126, -0.009099584, -0.017...",2,"{'level': 4, 'origin': 'summary_of_cluster_2_l..."
5,cluster_4_7,"[0.009915062, -0.09354369, 0.008022162, -0.004...",0,"{'level': 4, 'origin': 'summary_of_cluster_7_l..."
6,cluster_4_4,"[0.06272305, -0.090856984, 0.018282779, 0.0086...",2,"{'level': 4, 'origin': 'summary_of_cluster_4_l..."
7,cluster_4_1,"[0.03736072, -0.091609985, -0.005659424, -0.00...",2,"{'level': 4, 'origin': 'summary_of_cluster_1_l..."
8,cluster_4_8,"[0.012854084, -0.02482652, 0.045015525, -0.003...",2,"{'level': 4, 'origin': 'summary_of_cluster_8_l..."
9,cluster_4_5,"[0.022741336, -0.086818896, 0.0058439723, -0.0...",2,"{'level': 4, 'origin': 'summary_of_cluster_5_l..."


In [25]:
# 코사인 유사도
def cosine_similarity(vec1: np.ndarray, vec2: np.ndarray) -> float:
    """
    두 벡터 간 코사인 유사도를 계산합니다.
    두 벡터는 1차원 배열이어야 하며, 반환 값은 1 - cosine_distance (즉, 1에 가까울수록 유사도가 높음)
    """
    vec1 = np.array(vec1, dtype=np.float32).flatten()
    vec2 = np.array(vec2, dtype=np.float32).flatten()
    if vec1.shape[0] != vec2.shape[0]:
        raise ValueError(f"Dimension mismatch: {vec1.shape[0]} vs {vec2.shape[0]}")
    return 1 - cdist([vec1], [vec2], metric="cosine")[0][0]

# # raptor 검색
# def retrieve_progressive_documents(query_embedding: np.ndarray,
#                                     raptor_tree: dict,
#                                     neighbors_per_level: list = [16, 8, 4, 2, 1, 0],
#                                     similarity_threshold: float = 0.1) -> list:
#     results = []
#     q_vec = query_embedding.flatten()
#     base_similarity = 0.0

#     for level, n_neighbors in enumerate(neighbors_per_level):
#         if n_neighbors <= 0 or level not in raptor_tree:
#             continue

#         df = raptor_tree[level].copy()
#         embeddings_matrix = np.stack(df["embedding"].values)
#         similarities = 1 - cdist([q_vec], embeddings_matrix, metric="cosine")[0]
#         df["similarity"] = similarities
#         df = df[(df["similarity"] >= similarity_threshold) & (df["similarity"] < 1)]

#         if df.empty:
#             break

#         # 가장 높은 유사도 문서 추출
#         best_doc = df.loc[df["similarity"].idxmax()]
#         best_similarity = best_doc["similarity"]

#         if best_similarity >= base_similarity * 0.5 or base_similarity == 0:
#             base_similarity = best_similarity
#             best_doc["level"] = level
#             results.append(best_doc.to_dict())
#         else:
#             break  # 유사도 증가율 미달 → 탐색 중지

#     return results

def retrieve_cluster_ids_from_bottom(query_embedding: np.ndarray,
                                     raptor_tree: dict,
                                     similarity_threshold: float = 0) -> list:
    q_vec = query_embedding.flatten()
    max_level = max(raptor_tree.keys())

    # Step 1: Level 4에서 query_embedding 기준으로 유사한 노드 찾기
    if max_level not in raptor_tree:
        return []

    df = raptor_tree[max_level].copy()
    embeddings_matrix = np.stack(df["embedding"].values)
    similarities = 1 - cdist([q_vec], embeddings_matrix, metric="cosine")[0]
    df["similarity"] = similarities
    df = df[df["similarity"] >= similarity_threshold]
    df = df[~np.isclose(df["similarity"], 1.0)]  # 자기자신 제외

    if df.empty:
        return []

    # 초기 후보: Level 4 노드 정보 + 유사도 기준
    current_candidates = [
        {"current_embedding": row["embedding"], "threshold": row["similarity"]}
        for _, row in df.iterrows()
    ]

    # 상향 탐색: Level 3 → Level 1
    for level in reversed(range(1, max_level)):
        if level not in raptor_tree:
            return []  # 탐색 실패

        df_upper = raptor_tree[level].copy()
        embeddings_upper = np.stack(df_upper["embedding"].values)

        next_candidates = []
        for candidate in current_candidates:
            parent_vec = candidate["current_embedding"]
            threshold = candidate["threshold"]

            similarities = 1 - cdist([parent_vec], embeddings_upper, metric="cosine")[0]
            df_upper["similarity"] = similarities

            # 이전 유사도 이상만 유지 (유사도 유지 조건)
            df_filtered = df_upper[df_upper["similarity"] >= threshold]
            df_filtered = df_filtered[~np.isclose(df_filtered["similarity"], 1.0)]

            for _, parent_row in df_filtered.iterrows():
                next_candidates.append({
                    "current_embedding": parent_row["embedding"],
                    "threshold": parent_row["similarity"],
                    "level": level,
                    "node_id": parent_row.get("node_id")  # Level 1에서 클러스터 ID로 사용
                })

        if not next_candidates:
            return []  # 탐색 실패

        current_candidates = next_candidates  # 다음 탐색 준비

    # 최종 결과: Level 1 노드들의 node_id만 추출 (중복 제거)
    cluster_ids = set()
    for candidate in current_candidates:
        node_id = candidate.get("node_id")
        if node_id:
            cluster_ids.add(node_id)

    return list(cluster_ids)

def extract_nearest(n) :

    # Target User의 그래프 임베딩 가져오기
    query = np.array(data.iloc[n]['chunk_header_embedding'], dtype=np.float32)
    purchase_history=data.iloc[n]['movie_explain']

    # 쿼리 벡터를 FAISS 검색에 사용할 수 있도록 변환
    query_embedding = query.reshape(1, -1)  # FAISS는 2D 배열을 요구하므로 변환

    # 기존에 retrieve_documents_from_tree 함수를 사용하여 검색한 결과 (각 결과는 dict 형태)
    retrieved_docs = retrieve_cluster_ids_from_bottom(query_embedding, loaded_raptor_tree)

    # 리스트 초기화
    node_id = []
    level = []
    sim = []

    # 데이터프레임 생성
    for doc in retrieved_docs:
        node_id.append(doc.get("user_id", "N/A"))
        level.append(doc.get("level", "N/A"))
        sim.append(doc.get("similarity", 0))

    result_df = pd.DataFrame({'ID': node_id, 'Level': level, 'Similarity': sim}).nlargest(30, 'Similarity')

    return result_df, purchase_history

def classification_ids(result_df) :
    # 선택된 ID 리스트 (클러스터 ID와 개별 ID 혼합)
    selected_ids = result_df['ID'].tolist()

    return selected_ids

def summarize_individuals(individual_ids, vectorstore, purchase_history, embeddings):
    """
    각 개별 사용자에 대해, 해당 사용자의 여러 청크 중 purchase_history와
    코사인 유사도가 가장 높은 청크를 선택하여 요약하며, 본 영화 목록도 함께 반환함.
    """
    if isinstance(purchase_history, list):
        purchase_history = " ".join(purchase_history)  # 리스트 → 문자열 변환

    purchase_emb = embeddings.embed_documents([purchase_history])[0]  # 문서 임베딩

    individual_summaries = {}
    individual_movies = {}

    for uid in individual_ids:
        user_docs = [
            doc for doc in vectorstore.docstore._dict.values()
            if doc.metadata.get("UserId") == uid
        ]
        if user_docs:
            # 개별 사용자의 시청 영화 목록 가져오기
            user_movies = get_user_movies(user_docs)

            # 각 청크의 임베딩 계산
            doc_embeddings = embeddings.embed_documents([doc.page_content for doc in user_docs])

            # 가장 유사도가 높은 청크 선택
            best_doc_idx = max(range(len(user_docs)), key=lambda i: cosine_similarity(purchase_emb, doc_embeddings[i]))
            best_doc = user_docs[best_doc_idx]

            individual_summaries[uid] = best_doc.page_content
            individual_movies[uid] = user_movies[uid]  # 개별 사용자의 영화 목록 추가
        else:
            individual_summaries[uid] = f"{uid}: 해당 청크 없음"
            individual_movies[uid] = []

    return individual_summaries, individual_movies

In [18]:
# 개별 사용자 메타 청크 데이터와 합치기
def movie_summaries(individual_summaries, individual_movies):
    """
    클러스터에서 추출한 영화 요약과 개별 사용자의 요약을 하나로 병합합니다.
    """
    # 개별 사용자 요약을 하나의 텍스트로 병합
    final_summary = " ".join(individual_summaries.values())
    merged_movie_ids = {**individual_movies}
    return final_summary, merged_movie_ids

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# LangChain 프롬프트 생성
template = """
## item ##
Here is the summary of frequently watched movies from other users:
{record_summary}

## role ##
Based on the provided watching record summary ({record_summary}), recommend exactly 10 movies for the user to watch next.
You must prioritize movies that:
1. Appear frequently in the given watching record summary ({record_summary}).

For each recommended movie, provide a brief reason why it is recommended. The reason must include:
- How frequently the movie appears in the record summary.
- Why it might be appealing to the user based on similar preferences.

Provide the recommendations as a numbered list in the following format:
1. Movie ID - Reason
2. Movie ID - Reason
...
10. Movie ID - Reason
"""

def generate_answer(purchase_history, final_summary) :
    # OpenAI 모델 설정 (LangChain 최신 방식)
    response= chain.invoke({"purchase_history":purchase_history, "record_summary": final_summary })
    return response

# # 프롬프트 생성
# prompt = ChatPromptTemplate.from_template(template)
# llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
# chain = prompt | llm | StrOutputParser()

In [19]:
# OpenAI API 키 설정 (환경 변수에서 가져오기)
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
# 테스트 데이터 불러오기
df_test = pd.read_csv('./data/test_movie.csv')
df_test.head()

,UserId,movie_explain
0,1,"[""48 (Animation|Children's|Musical|Romance) ra..."
1,2,['1917 (Action|Adventure|Sci-Fi|Thriller) rati...
2,3,"[""2081 (Animation|Children's|Comedy|Musical|Ro..."
3,4,['1954 (Action|Drama) ratings: 5']
4,5,['288 (Action|Thriller) ratings: 2']


In [20]:
!pip install torch_geometric

Defaulting to user installation because normal site-packages is not writeable


In [16]:
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
import re

def get_user_movies(data):
    user_movies = defaultdict(list)

    for doc in data:
        user_id = doc.metadata['UserId']
        page_content = doc.page_content

        # 정규식을 사용하여 영화 ID 추출 (ratings 이전 내용만)
        movie_ids = re.findall(r'(\d+)(?= \()', page_content)

        # 사용자별로 영화 ID 추가
        user_movies[user_id].extend(movie_ids)

    return user_movies

def create_user_movie_graph(user_movies):
    # 그래프 생성
    G = nx.Graph()

    # 사용자와 영화 노드 추가 및 엣지 생성
    for user_id, movies in user_movies.items():
        user_node = f"User {user_id}"
        G.add_node(user_node, type='user')

        for movie_id in movies:
            movie_node = f"Movie {movie_id}"
            G.add_node(movie_node, type='movie')
            G.add_edge(user_node, movie_node)

    return G


def visualize_graph(G):
    # 그래프 시각화
    plt.figure(figsize=(20, 15))

    # 사용자와 영화 노드 분리
    users = [node for node in G.nodes() if G.nodes[node]['type'] == 'user']
    movies = [node for node in G.nodes() if G.nodes[node]['type'] == 'movie']

    # 레이아웃 설정
    pos = nx.spring_layout(G, k=0.5, iterations=50)

    # 노드 그리기
    nx.draw_networkx_nodes(G, pos, nodelist=users, node_color='lightblue', node_size=300, alpha=0.8)
    nx.draw_networkx_nodes(G, pos, nodelist=movies, node_color='lightgreen', node_size=200, alpha=0.8)

    # 엣지 그리기
    nx.draw_networkx_edges(G, pos, alpha=0.5)

    # 레이블 그리기
    nx.draw_networkx_labels(G, pos, font_size=8, font_weight="bold")

    plt.title("User-Movie Relationship Graph", fontsize=16)
    plt.axis('off')
    plt.tight_layout()

    # 그래프 저장
    plt.savefig('user_movie_graph.png', dpi=300, bbox_inches='tight')
    plt.close()

import re

# 과거 기록에서 영화 ID 추출
def extract_previous_movie_ids(purchase_history):
    # 영화 ID만 추출 (숫자와 괄호 전까지만 가져옴)
    previous_movies = re.findall(r'(\d+)(?=\s\()', ' '.join(purchase_history))
    return set(previous_movies)

# 동시 시청 영화에서 이전 기록을 제외하는 함수
def filter_movies_by_history(top_movies, previous_movie_ids):
    # 영화 ID만 추출하여 비교 후 제외
    filtered_movies = [(movie, count) for movie, count in top_movies if movie.split()[1] not in previous_movie_ids]
    return filtered_movies
def get_top_10_common_movies(G):
    # 영화 노드만 필터링
    movies = [node for node in G.nodes() if G.nodes[node]['type'] == 'movie']

    # 영화별 연결된 사용자 수 계산
    movie_view_counts = {movie: len(list(G.neighbors(movie))) for movie in movies}

    # 사용자 수 기준으로 내림차순 정렬하여 상위 10개 추출
    top_10_movies = sorted(movie_view_counts.items(), key=lambda x: x[1], reverse=True)[:20]

    # 결과 반환
    return top_10_movies

In [26]:
test_answer = df_test.iloc[1]['movie_explain']
movie_name = next((word for word in test_answer.split() if word.isdigit()), None)

result_df, purchase_history = extract_nearest(1)
result_df
# individual_ids = classification_ids(result_df)

# # # 클러스터 & 개별 사용자 데이터를 한 번에 캐싱하여 처리
# all_docs = list(new_vectorstore.docstore._dict.values())

# # # 클러스터 및 개별 사용자의 요약 및 시청 영화 ID 정보 가져오기
# individual_summaries, individual_movies = summarize_individuals(individual_ids, new_vectorstore, purchase_history, embeddings)

# # # 최종 요약과 사용자의 시청 영화 ID 목록 병합
# final_summary, merged_movie_ids = movie_summaries(individual_summaries, individual_movies)

# final_summary

,ID,Level,Similarity


In [16]:
import torch
from tqdm import tqdm

results = []

# 전체 문서 한 번만 로드하여 캐싱
all_docs = list(new_vectorstore.docstore._dict.values())

# # 사용자-영화 관계 데이터 미리 생성
# user_movies = get_user_movies(all_docs)

# # PyTorch Geometric 그래프 생성 (GPU 사용)
# G = create_user_movie_graph_fast(user_movies)

# 1부터 6040까지 반복하면서 진행률 표시 (tqdm 사용)
for idx in tqdm(range(1000), desc="Processing users", unit="user"):

    # 정답 추출
    test_answer = df_test.iloc[idx]['movie_explain']

    # movie_name 추출
    try:
        movie_name = re.search(r"\d+", test_answer).group()
    except AttributeError:
        movie_name = None  # 매칭 실패 시 None으로 처리

    # 쿼리에 따른 최근접 이웃 추출 + 쿼리 사용자의 최신 상호작용 추출
    result_df, purchase_history = extract_nearest(idx)
    individual_ids = classification_ids(result_df)

    # 클러스터 & 개별 사용자 데이터를 한 번에 캐싱하여 처리
    all_docs = list(new_vectorstore.docstore._dict.values())

    # 클러스터 및 개별 사용자의 요약 및 시청 영화 ID 정보 가져오기
    individual_summaries, individual_movies = summarize_individuals(individual_ids, new_vectorstore, purchase_history, embeddings)
    
    # 최종 요약과 사용자의 시청 영화 ID 목록 병합
    final_summary, merged_movie_ids = movie_summaries(individual_summaries, individual_movies)

    # 사용자-영화 그래프 생성
    user_movies = get_user_movies(new_vectorstore.docstore._dict.values())

    G = create_user_movie_graph(user_movies)

    # 이용자의 과거 시청 영화 목록 가져오기
    previous_movie_ids = extract_previous_movie_ids(purchase_history)

    # Top-10 가장 많이 본 영화 가져오기
    top_movies = get_top_10_common_movies(G)

    # 과거 시청한 영화 제외 후 추천 영화 필터링
    filtered_movies = filter_movies_by_history(top_movies, previous_movie_ids)

    # 리스트 내부 영화 정보를 문자열로 변환 후 정규식 적용
    movie_ids = [re.search(r'(\d+)', movie[0]).group() for movie in filtered_movies if re.search(r'(\d+)', movie[0])]

    # movie_name이 movie_titles에 포함되는지 여부
    is_included = movie_name in movie_ids if movie_name else False

    # 결과 저장
    results.append({
        "movie_title": movie_ids,
        "movie_name": movie_name,
        "is_included": is_included
    })

# 결과를 DataFrame으로 변환
final_results = pd.DataFrame(results)
final_results.head()

final_results.to_csv('results_ex1.csv', index=False)

Processing users:   0%|          | 0/1000 [00:00<?, ?user/s]/tmp/ipykernel_353550/1365346341.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_doc["level"] = level
/tmp/ipykernel_353550/1365346341.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_doc["level"] = level
Processing users:   0%|          | 1/1000 [00:02<49:44,  2.99s/user]/tmp/ipykernel_353550/1365346341.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b

In [17]:
# Hit@k 계산 (k=10)
k = 10
final_results['hit'] = final_results.apply(lambda row: row['movie_name'] in row['movie_title'][:k], axis=1)

# 전체 Hit@k 계산
hit_at_k = final_results['hit'].mean()  # Hit@k 비율
print(f"Hit@{k}: {hit_at_k}")

Hit@10: 0.036
